
<h1 id="Aprendizado-de-Máquina-2018.2">Aprendizado de Máquina 2018.2<a class="anchor-link" href="#Aprendizado-de-Máquina-2018.2">¶</a></h1>



<h3 id="Trabalho-1---Estimando-o-preço-de-imóveis-com-técnicas-de-regressão-(competição-do-Kaggle)">Trabalho 1 - Estimando o preço de imóveis com técnicas de regressão (competição do Kaggle)<a class="anchor-link" href="#Trabalho-1---Estimando-o-preço-de-imóveis-com-técnicas-de-regressão-(competição-do-Kaggle)">¶</a></h3>



<p>Aluno: Felipe Ferreira da Silva</p>
<p>Nome de UsuÃ¡rio no Kaggle: <strong>felipefdsilva</strong></p>



<h3 id="Introdução">Introdução<a class="anchor-link" href="#Introdução">¶</a></h3>



<p>Este trabalho tem como propÃ³sito a participaÃ§Ã£o na competiÃ§Ã£o Kaggle: "House Prices: Advanced Regression Techniques". Mais detalhes sobre a competiÃ§Ã£o podem ser lidos no endereÃ§o: <a href="https://www.kaggle.com/c/house-prices-advanced-regression-techniques">https://www.kaggle.com/c/house-prices-advanced-regression-techniques</a></p>



<p>Em resumo, o dataset foi manipulado de forma a:</p>
<ol>
<li>remover outliers, </li>
<li>normalizar a distribuiÃ§Ã£o de valores da variÃ¡vel alvo</li>
<li>normalizar a distribuiÃ§Ã£o de preditores </li>
<li>minimizar o nÃºmero de colunas </li>
<li>prencher campos vazios ("NaN") com o valor mÃ©dio</li>
<li>tratar variÃ¡veis categÃ³ricas atravÃ©s de one hot encoding.</li>
</ol>



<p>Foram utilizados algumas os regressores:</p>
<ol>
<li>LinearRegression()</li>
<li>RandomForest()</li>
<li>GradientBoostingRegressor()</li>
<li>ElasticNet</li>
</ol>
<p>Com o Ãºltimo apresentando o melhor resultado.</p>



<p>O modelo foi testado atravÃ©s de regressÃ£o linear. VÃ¡rios grÃ¡ficos sÃ£o exibidos ao longo do relatÃ³rio para ilustrar e justificar as operaÃ§Ãµes realizadas.</p>



<p>O relatÃ³rio foi escrito com auxilio da ferramenta Jupyter Notebook. Todo o cÃ³digo Python utilizado encontra-se aqui, com comentÃ¡rios explicativos para cada cÃ©lula.</p>



<h3 id="Importação-das-bibliotecas-necessárias">Importação das bibliotecas necessárias<a class="anchor-link" href="#Importação-das-bibliotecas-necessárias">¶</a></h3>



<p>Foram utilizadas basicamente bibliotecas relacionada a plotagem de grÃ¡ficos, ferramentas matemÃ¡ticas e de estatÃ­stica, e bibliotecas scikit-learn para manipulaÃ§Ã£o do dataset e criaÃ§Ã£o dos modelos.</p>


In [ ]:

import pandas as pd
import numpy as np
from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.special import boxcox1p
from sklearn.preprocessing import Imputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor




<p>ConfiguraÃ§Ãµes para o grÃ¡ficos</p>


In [ ]:

plt.style.use(style='ggplot')




<h3 id="Importando-os-dados-e-convertendo-num-dataset-do-pandas">Importando os dados e convertendo num dataset do pandas<a class="anchor-link" href="#Importando-os-dados-e-convertendo-num-dataset-do-pandas">¶</a></h3>


In [ ]:

train_file = './train.csv'
test_file = './test.csv'

train_dataset = pd.read_csv(train_file)
test_dataset = pd.read_csv(test_file)

target = train_dataset.SalePrice




<p>Com o cÃ³digo da cÃ©lula abaixo, observa-se a formato dos dataset de treino e teste.</p>


In [ ]:

print (train_dataset.shape)
print (test_dataset.shape)




<p>Observando o conteÃºdo de algumas colunas:</p>


In [ ]:

train_dataset.head()




<h3 id='Estudando-a-variavel-alvo-"SalePrice"'>Estudando a variavel alvo <strong>"SalePrice"</strong><a class="anchor-link" href='#Estudando-a-variavel-alvo-"SalePrice"'>¶</a></h3>


In [ ]:

sns.distplot(target, color='blue');
print("Skewness:", target.skew())




<p>Observa-se pelo grÃ¡fico acima que a variÃ¡vel alvo (SalePrice) tem uma cauda longa para a direita. Modelos de regressÃ£o linear tem melhor comportamento quando trabalham com variÃ¡veis de distribuiÃ§Ã£o normal. Portanto, fazemos uma trasformaÃ§Ã£o logarÃ­timica para aproximar SalePrice de uma gaussiana.</p>


In [ ]:

target = np.log(target)
sns.distplot(target, color='blue');
print("Skewness: ", target.skew())




<h3 id="Lidando-com-variáveis-com-campos-vazios">Lidando com variáveis com campos vazios<a class="anchor-link" href="#Lidando-com-variáveis-com-campos-vazios">¶</a></h3>



<p>Observa-se com o cÃ³digo abaixo as caracterÃ­sticas que possuem mais campos vazios no dataset, ordenadas de cima para baixo de acordo com o percentual de dados ausentes.</p>


In [ ]:

total = train_dataset.isnull().sum().sort_values(ascending=False)
percent = (train_dataset.isnull().sum()/train_dataset.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)




<p>Escolhe-se descartar todas as colunas que possuem mais de 15% de dados faltando, por considerar este um valor suficientemente alto para desconsiderar o potencial preditivo dessas caracterÃ­sticas.</p>


In [ ]:

train_dataset = train_dataset.drop((missing_data[missing_data['Total'] > 81]).index, axis=1)
test_dataset = test_dataset.drop((missing_data[missing_data['Total'] > 81]).index, axis=1)




<h3 id="Lidando-com-variáveis-numéricas">Lidando com variáveis numéricas<a class="anchor-link" href="#Lidando-com-variáveis-numéricas">¶</a></h3>


In [ ]:

numeric_features = train_dataset.select_dtypes(include=[np.number])




<p>Um vez destacada a parcela do dataset que contÃ©m colunas de dados numÃ©ricas, estuda-se a correlaÃ§Ã£o destas colunas com a variavel alvo.</p>


In [ ]:

corr = numeric_features.corr()

print (corr['SalePrice'].sort_values(ascending=False)[1:])




<p>De acordo com a descriÃ§Ã£o do dataset, "MSSubClass" Ã© uma variÃ¡vel categÃ³rica e nÃ£o numÃ©rica. Portanto, precisa de um cuidado especial. Por enquanto, serÃ¡ retirada do dataset numeric_features.</p>
<p>TambÃ©m serÃ£o retiradas as colunas 'SalePrice' e 'Id', para realizaÃ§Ã£o de plotagens.</p>


In [ ]:

numeric_features = numeric_features.drop(['SalePrice', 'MSSubClass','Id'], axis=1)
numeric_features_list = numeric_features.columns
numeric_features.shape




<p>Com os grÃ¡ficos abaixo, observa-se a distribuiÃ§Ã£o das medidas de cada variÃ¡vel, com o objetivo de encontrar outliers (pontos isolados que destoam da tendÃªncia da populaÃ§Ã£o e que tem potencial de prejudicar a modelagem).</p>


In [ ]:

fig, axes = plt.subplots(figsize=(15, 55))
sns.set()
for i in range(1, 34):
    plt.subplot(12, 3, i)
    sns.distplot(numeric_features[numeric_features_list[i-1]].dropna())
plt.show()



In [ ]:

fig, axes = plt.subplots(figsize=(22, 60))
sns.set()
for i in range(1, 34):
    plt.subplot(12, 3, i)
    sns.scatterplot(y='SalePrice', x=numeric_features[numeric_features_list[i-1]], data=train_dataset)
plt.show()




<p>Observando os grÃ¡ficos acima, decide-se por retirar colunas que possuem massiva concentraÃ§Ã£o de zeros, pois se a variÃ¡vel tem comportamento constante, nÃ£o influenciarÃ¡ no modelo.</p>


In [ ]:

train_dataset = train_dataset.drop(['BsmtFinSF2', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'LowQualFinSF', 'KitchenAbvGr'], axis=1)
test_dataset = test_dataset.drop(['BsmtFinSF2', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'LowQualFinSF', 'KitchenAbvGr'], axis=1)



In [ ]:

numeric_features = numeric_features.drop(['BsmtFinSF2', '3SsnPorch', 'ScreenPorch','PoolArea', 'MiscVal', 'LowQualFinSF', 'KitchenAbvGr'], axis=1)




<p>Observa-se em escala maior algumas variÃ¡veis em que se faz necessÃ¡ria a remoÃ§Ã£o de <em>outliers</em>.</p>



<h5 id="TotalBsmt">TotalBsmt<a class="anchor-link" href="#TotalBsmt">¶</a></h5>


In [ ]:

sns.scatterplot(y='SalePrice', x='TotalBsmtSF', data=train_dataset)




<h5 id="LotArea">LotArea<a class="anchor-link" href="#LotArea">¶</a></h5>


In [ ]:

sns.scatterplot(y='SalePrice', x='LotArea', data=train_dataset)




<h5 id="GrLivArea">GrLivArea<a class="anchor-link" href="#GrLivArea">¶</a></h5>


In [ ]:

sns.scatterplot(y='SalePrice', x='GrLivArea', data=train_dataset);




<h5 id="GarageArea">GarageArea<a class="anchor-link" href="#GarageArea">¶</a></h5>


In [ ]:

sns.scatterplot(y='SalePrice', x='GarageArea', data=train_dataset);




<p>Remove-se entÃ£o, os outliers com as operaÃ§Ãµes abaixo:</p>


In [ ]:

train_dataset = train_dataset[train_dataset['TotalBsmtSF'] < 2500]
train_dataset = train_dataset[train_dataset['LotArea'] < 100000]
train_dataset = train_dataset[train_dataset['GrLivArea'] < 3000]
train_dataset = train_dataset[train_dataset['GarageArea'] < 1000]
train_dataset = train_dataset[train_dataset['1stFlrSF'] < 2300]




<p>A etapa abaixo Ã© uma parte do trabalho em que nÃ£o houve sucesso. Se buscou transformar alguns preditores para aproximÃ¡-los tambÃ©m da gaussiana. As transformaÃ§Ãµes se resumiram as colunas com assimetria maior que 75%. Utilizou-se a transformaÃ§Ã£o de Box-cox, com lambda escolhido empiricamente. PorÃ©m, as previsÃµes do modelo com tais mediÃ§Ãµes geraram erros maiores quando submetidas no Kaggle.</p>


In [ ]:

skewed_feats = numeric_features.apply(lambda x: x.dropna().skew()).sort_values(ascending=False)
print("\nSkew in features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness



In [ ]:

skewed_features = list(skewness[skewness['Skew'] > 0.75].index)
lmbda = 0.15
print ("Feature", "\t", "Original Skewness", "\t", "Skewness after Box-cox transformation\n")
for feat in skewed_features:
    print (feat, "\t", numeric_features[feat].skew(), "\t", (boxcox1p(numeric_features[feat], lmbda)).skew())




<p>Como alguns colunas se distanciaram a distribuiÃ§Ã£o normal, estas foram retiradas da transformaÃ§Ã£o, pois entende-se que a transformaÃ§Ã£o, nestes casos, prejudicava o modelo.</p>


In [ ]:

for col in ['BsmtHalfBath', 'EnclosedPorch', 'TotalBsmtSF', 'BsmtUnfSF']:
    skewed_features.remove(col)

for feat in skewed_features:
#    train_dataset[feat] = boxcox1p(numeric_features[feat], lmbda)
#    test_dataset[feat] = boxcox1p(numeric_features[feat], lmbda)
    print (feat, "\t", numeric_features[feat].skew(), "\t", (boxcox1p(numeric_features[feat], lmbda)).skew())




<p>Retornando a coluna "MSSubClass", que deve ser transformada em coluna de dados categÃ³ricos para ser tratada corretamente. Foram criadas siglas de acordo com a descriÃ§Ã£o do dataset para cada valor numÃ©rico. Cada sigla agora Ã© uma categoria de MSSubClass.</p>


In [ ]:

#20 	1-STORY 1946 & NEWER ALL STYLES  	(NAL)
#30 	1-STORY 1945 & OLDER  	(OLD)
#40 	1-STORY W/FINISHED ATTIC ALL AGES  	(SFAAT)
#45 	1-1/2 STORY - UNFINISHED ALL AGES  	(SUAA)
#50 	1-1/2 STORY FINISHED ALL AGES (SFAA)
#60 	2-STORY 1946 & NEWER  	(SN)
#70 	2-STORY 1945 & OLDER  	(SO)
#75 	2-1/2 STORY ALL AGES  	(SAG)
#80 	SPLIT OR MULTI-LEVEL  	(SML)
#85 	SPLIT FOYER  	(SF)
#90 	DUPLEX - ALL STYLES AND AGES  	(DASA)
#120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER  	(OneSPUD)
#150	1-1/2 STORY PUD - ALL AGES  	(HalfSPUD)
#160	2-STORY PUD - 1946 & NEWER  	(TwoSPUD)
#180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER  	(PUDMultilvl)
#190	2 FAMILY CONVERSION - ALL STYLES AND AGES 	(FamConv)

def categorize_MSSubClass(x):
    if x == 20:
        return 'NAL'
    if x == 30:
        return 'OLD'
    if x == 40:
        return 'SFAAT'
    if x == 45:
        return 'SUAA'
    if x == 50:
        return 'SFAA'
    if x == 60:
        return 'SN'
    if x == 70:
        return 'SO'
    if x == 80:
        return 'SML'
    if x == 85:
        return 'SF'
    if x == 90:
        return 'DASA'
    if x == 120:
        return 'OneSPUD'
    if x == 150:
        return 'HalfSPUD'
    if x == 160:
        return 'TwoSPUD'
    if x == 180:
        return 'PUDMultilvl'
    if x == 190:
        return 'FamConv'
    
train_dataset['cat_MSSubClass'] = train_dataset.MSSubClass.apply(categorize_MSSubClass)
test_dataset['cat_MSSubClass'] = test_dataset.MSSubClass.apply(categorize_MSSubClass)

test_dataset['cat_MSSubClass'].value_counts()




<p>A saÃ­da acima confirma a transformaÃ§Ã£o, com a crianÃ§Ã£o da nova coluna "cat_MSSubClass".</p>



<h3 id="Lidando-com-variáveis-categóricas">Lidando com variáveis categóricas<a class="anchor-link" href="#Lidando-com-variáveis-categóricas">¶</a></h3>



<p>Em suma, apenas observa-se o comportamento das variÃ¡veis e estuda-se como mapeÃ¡-las em valores numÃ©ricos.</p>


In [ ]:

categoric_features = train_dataset.select_dtypes(include='object')
categoric_features_list = categoric_features.columns
print ("Shape:", categoric_features.shape, '\n')
print ("Estas sÃ£o as colunas categÃ³ricas: ", categoric_features.columns,"\n")




<p>Na cÃ©lula abaixo, adiciona-se a categoria "MISSING" para que seja possÃ­vel gerar grÃ¡ficos de todas as variÃ¡veis caategÃ³ricas, inclusive as que possuem valores NaN.</p>


In [ ]:

for feat in categoric_features_list:
    categoric_features[feat] = categoric_features[feat].astype('category')
    if categoric_features[feat].isnull().any():
        categoric_features[feat] = categoric_features[feat].cat.add_categories(['MISSING'], inplace=False)
        categoric_features[feat] = categoric_features[feat].fillna('MISSING')



In [ ]:

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)

f = pd.melt(train_dataset, id_vars = ['SalePrice'], value_vars=categoric_features_list)
g = sns.FacetGrid(f, col="variable",  col_wrap=3, sharex=False, sharey=False, height=5)
g = g.map(boxplot, "value", "SalePrice")




<p>Observa-se que a coluna "SaleCondition" em especial, existe um valor('Partial') que destoa do restante, possuindo uma relaÃ§Ã£o com preÃ§os mais elevados. Portanto, decide-se codificar a coluna como se segue, convertendo os valores "Partial" em '1's e o restantes em zeros.</p>


In [ ]:

def encode(x): return 1 if x == 'Partial' else 0
train_dataset['enc_SaleCondition'] = train_dataset.SaleCondition.apply(encode)
test_dataset['enc_SaleCondition'] = test_dataset.SaleCondition.apply(encode)




<p>JÃ¡ no caso de Street, obeserva-se que a mesma Ã© uma variÃ¡vel binÃ¡ria (a rua Ã© pavimentada ou nÃ£o). Portanto, codifica-se como se segue:</p>


In [ ]:

train_dataset['enc_street'] = pd.get_dummies(train_dataset.Street, drop_first=True)
test_dataset['enc_street'] = pd.get_dummies(test_dataset.Street, drop_first=True)

train_dataset['enc_street'].value_counts()




<p>Pode-se realizar o mesmo procedimento para "Utilities" (uma variÃ¡vel que indica as utilidades suportadas pela casa: gÃ¡s, eletricidade e Ã¡gua). PorÃ©m, observando a distribuiÃ§Ã£o de seus valores, chega-se a conclusÃ£o que Ã© uma coluna descatÃ¡vel.</p>


In [ ]:

print(train_dataset['Utilities'].value_counts(), '\n')
print(test_dataset['Utilities'].value_counts())




<p>Nota-se que, no dataset de treino sÃ³ hÃ¡ um valor "NoSeWa"(Eletricidade e GÃ¡s apenas). No dataset de teste sÃ³ hÃ¡ casa com todas as utilidades (AllPub). Portanto, 'Utilities' pode ser descartado.</p>


In [ ]:

train_dataset = train_dataset.drop(['Utilities'], axis = 1)
test_dataset = test_dataset.drop(['Utilities'], axis = 1)



In [ ]:

print(train_dataset.shape)
print(test_dataset.shape)




<p>Criamos colunas binÃ¡rias para o restante das colunas categÃ³ricas, mapeando cada categoria em uma nova coluna binÃ¡ria.</p>


In [ ]:

ohe_train_dataset = pd.get_dummies(train_dataset)
ohe_test_dataset = pd.get_dummies(test_dataset)



In [ ]:

print(ohe_train_dataset.shape)
print(ohe_test_dataset.shape)




<p>O Comando abaixo alinhas os dois datasets, para que regressor receba as mesmas colunas e na mesma ordem nos dois casos.</p>



<h3 id="Ajuste-final">Ajuste final<a class="anchor-link" href="#Ajuste-final">¶</a></h3>



<p>Por fim, iremos remover todas as colunas que possuem correlaÃ§Ã£o menor que 7% com a variÃ¡vel alvo. Este valor foi escolhido empiricamente.</p>


In [ ]:

drop_list = []
for key, value in ohe_train_dataset.corr()['SalePrice'].items():
    if abs(value) < 0.07 and key != "Id":
        drop_list.append(key)

final_train_dataset = ohe_train_dataset.drop(drop_list, axis=1)



In [ ]:

y = np.log(final_train_dataset.SalePrice)
X = final_train_dataset.drop(['SalePrice'], axis=1)



In [ ]:

X_train, X_test = X.align(ohe_test_dataset, join='left', axis=1)



In [ ]:

print(X_train.shape)
print(X_test.shape)




<h3 id="Construindo-o-modelo">Construindo o modelo<a class="anchor-link" href="#Construindo-o-modelo">¶</a></h3>



<p>Representamos os preditores usados para regressÃ£o como o vetor X e a variÃ¡vel alvo como y.</p>
<p>Utilizaremos o regressor ElasticNet, pois este foi o que apresentou melhor resultado. O segundo melhor resultado foi obtido com o regressor linear. TambÃ©m foram testados os regressores Floresta RandÃ´mica e Gradient Boosting, que apresentaram resultados mais distantes dos dois primeiros.</p>


In [ ]:

my_pipeline = make_pipeline(Imputer(), ElasticNet(alpha=0.0005, l1_ratio=.9))




<p>Retirando a coluna 'Id' de X_train para o teste de validaÃ§Ã£o cruzada. A coluna 'Id' serÃ¡ retirada de X_test algumas cÃ©lulas abaixo.</p>


In [ ]:

X_train = X_train.drop(['Id'], axis=1)




<h3 id="Testando-o-modelo-com-validação-cruzada-e-RMSE-e-MAE">Testando o modelo com validação cruzada e RMSE e MAE<a class="anchor-link" href="#Testando-o-modelo-com-validação-cruzada-e-RMSE-e-MAE">¶</a></h3>



<p>A funÃ§Ã£o cross_val_score do scikit-learn trabalha apenas com erro quadrÃ¡tico mÃ©dio negativo, assim como erro absoluto mÃ©dio negativo. Portanto, para termos o RMSE, Ã© preciso realizar algumas operaÃ§Ãµes matemÃ¡ticas.</p>


In [ ]:

scores = [sqrt(-1*x) for x in cross_val_score(my_pipeline, X_train, y, scoring='neg_mean_squared_error')]
print(scores,'\n')

print ("MÃ©dia:", np.mean(scores))




<p>O erro absoluto mÃ©dio Ã© calculado abaixo:</p>


In [ ]:

scores = cross_val_score(my_pipeline, X_train, y, scoring='neg_mean_absolute_error')
print(scores*(-1),'\n')

print ("MÃ©dia:", np.mean(scores)*(-1))




<h3 id="Fazendo-as-previsões-com-o-arquivo-de-teste-e-gerando-o-arquivo-csv-para-submissão">Fazendo as previsões com o arquivo de teste e gerando o arquivo csv para submissão<a class="anchor-link" href="#Fazendo-as-previsões-com-o-arquivo-de-teste-e-gerando-o-arquivo-csv-para-submissão">¶</a></h3>



<p>Criando o dataset "submission" que terÃ¡ apenas duas colunas: "Id" e "SalePrice"</p>


In [ ]:

submission = pd.DataFrame()
submission['Id'] = X_test.Id




<p>Retirando a coluna "Id" do dataset de teste para realizar as previsÃµes.</p>


In [ ]:

X_test = X_test.drop(['Id'], axis=1)




<p>Preenchendo os campos vazios do dataset de teste.</p>


In [ ]:

my_imputer = Imputer()
my_imputer.fit_transform(X_train)
final_test = my_imputer.transform(X_test)




<p>Gerando as previsÃµes</p>


In [ ]:

my_pipeline.fit(X_train, y)
predictions = my_pipeline.predict(final_test)




<p>Aplicando a exponencial em 'SalePrice' para anular a transformaÃ§Ã£o feita anteriormente.</p>


In [ ]:

final_predictions = np.exp(predictions)
submission['SalePrice'] = final_predictions




<p>Finalmente, na cÃ©lula abaixo, gera-se o arquivo que serÃ¡ submetido no Kaggle.</p>


In [ ]:

submission.to_csv('submission.csv', index=False)

